In [1]:
import numpy as np
import torch
import torchattacks
from deepcase_copy.context_builder import ContextBuilder
from deepcase_copy.preprocessing import Preprocessor

builder = ContextBuilder.load('save/builder.save')
preprocessor = Preprocessor(
    length=10,  # 10 events in context
    timeout=86400,  # Ignore events older than 1 day (60*60*24 = 86400 seconds)
)
context, events, labels, mapping = preprocessor.csv('alerts.csv', verbose=True)

# In case no labels are provided, set labels to -1
# IMPORTANT: If no labels are provided, make sure to manually set the labels
# before calling the interpreter.score_clusters method. Otherwise, this will
# raise an exception, because scores == NO_SCORE cannot be computed.
if labels is None:
    labels = np.full(events.shape[0], -1, dtype=int)

# Cast to cuda if available
if torch.cuda.is_available():
    builder = builder.to('cuda')
    events = events.to('cuda')
    context = context.to('cuda')


# Split into train and test sets (20:80) by time - assuming events are ordered chronologically
events_train  = events [:events.shape[0]//5 ]
events_test   = events [ events.shape[0]//5:]

context_train = context[:events.shape[0]//5 ]
context_test  = context[ events.shape[0]//5:]

labels_train  = labels [:events.shape[0]//5 ]
labels_test   = labels [ events.shape[0]//5:]

Loading: 100%|██████████| 3353/3353 [00:00<00:00, 3442.40it/s]


In [2]:
l = 1

chosen_index = 0
context_picked = context[chosen_index:chosen_index+l]
events_picked = events[chosen_index:chosen_index+l]
labels_picked = labels[chosen_index:chosen_index+l]
# confidence, attention = builder.predict(context_picked)
# topk = torch.topk(confidence[0][0], 1)
# context_picked, events_picked, attention[0][0], topk
# for i in range(l):
#     if labels_picked[i] not in [1,2,3]:
#         print(labels_picked[i], context_picked[i], events_picked[i])

In [6]:
# cim = builder.embedding_one_hot(context_picked)
# cim.requires_grad_(True)
# output = builder.predict(cim)

In [14]:
from deepcase_copy.context_builder.loss import LabelSmoothing

def max_to_one(tensor):
    max_indices = torch.argmax(tensor, dim=-1, keepdim=True)
    result = torch.zeros_like(tensor)
    result.scatter_(-1, max_indices, 1.0)
    return result

def print_results(results, i):
    results_picked = torch.topk(results[0][0][0], 3)
    exp = results_picked.values.exp()
    indices = results_picked.indices
    s = ""
    for j in range(3):
        s += f"[{indices[j].item()}] {round(exp[j].item(), 3)}, "
    print(f"Prediction {i}: {s}")
    return indices

def compute_change(trace, original, epsilon=0.1):
    a = torch.clamp(original - epsilon, min=0)
    b = (trace >= a).float() * trace + (trace < a).float() * a
    c = (b > original + epsilon).float() * (original + epsilon) + (b <= original + epsilon).float() * b
    return torch.clamp(c, max=1).detach()

def bim_attack(context_given, target_given, alpha=0.1, epsilon=0.1, num_iterations=10):
    change = None
    original_context = builder.embedding_one_hot(context_given)
    context_processed = builder.embedding_one_hot(context_given)
    criterion = LabelSmoothing(builder.decoder_event.out.out_features, 0.1)    
    
    for i in range(num_iterations):
        context_processed.requires_grad_(True)
        output = builder.predict(context_processed)
        indices = print_results(output, i)
        if events_picked[0] not in indices:
            break
        loss = criterion(output[0][0], events_picked)
        loss.backward()
        grad = context_processed.grad.sign()
        if change is None:
            change = alpha * grad
        else:
            change += alpha * grad
        context_processed = context_processed + change
        context_processed = compute_change(context_processed, original_context, epsilon)
        
bim_attack(context_given=context_picked, target_given=events_picked, alpha=0.1, epsilon=0.1, num_iterations=10)

Prediction 0: [64] 0.341, [72] 0.282, [24] 0.143, 
Prediction 1: [64] 0.724, [24] 0.125, [57] 0.04, 


In [17]:
# import torch
# import torch.nn as nn
# class SimpleModel(nn.Module):
#     def __init__(self, input_size, output_size):
#         super(SimpleModel, self).__init__()
#         self.fc = nn.Linear(input_size, output_size)
#     
#     def forward(self, x):
#         return self.fc(x)
# 
# input_size = 270 
# output_size = 1
# model = SimpleModel(input_size, output_size)
# int_input = torch.tensor([[2, 3, 5]], dtype=torch.int64)  # Integer-valued input
# 
# def one_hot_encode(int_tensor, num_classes):
#     return torch.nn.functional.one_hot(int_tensor, num_classes).float()
# 
# one_hot_input = one_hot_encode(int_input, 90).view(1, -1)  
# one_hot_input.requires_grad_(True)
# target = torch.tensor([[10.0]], dtype=torch.float32)
# criterion = nn.MSELoss()
# output = model(one_hot_input)
# loss = criterion(output, target)
# loss.backward()
# print(output, target)
# print(one_hot_input.grad)


tensor([[-0.1143]], grad_fn=<AddmmBackward0>) tensor([[10.]])
tensor([[-0.1077, -0.1853,  1.0463, -0.4446,  0.1205, -0.6067, -0.1476,  0.3568,
         -0.1219,  1.1074,  0.1277, -0.7452, -0.7724, -0.6487, -0.0372,  1.0985,
          0.7424,  0.3551,  0.9772, -0.5586,  1.2131,  1.0179, -1.1229, -0.3037,
          0.8177,  0.8633,  1.0296, -0.6272, -0.4376, -0.7597,  0.0471, -0.4535,
          1.0059, -1.1586,  0.1231,  0.8764, -0.3022, -0.3063, -0.6398,  1.0514,
         -1.2093,  1.0124, -0.9577, -0.1990,  0.5465,  0.1698, -0.8128,  0.5603,
          0.9736,  0.6556,  0.0100, -0.7559,  0.6390,  0.2965, -0.6951, -0.7969,
         -0.7412, -0.0679, -0.8584,  0.4403, -0.4508,  0.9497,  0.6262, -0.8821,
          0.5530, -0.0623, -0.0568,  0.7504,  0.1850,  0.3488, -0.0998,  0.3175,
          0.0525,  0.4194,  1.1491,  0.1296,  1.1290,  1.0317,  0.2275,  0.8821,
          0.2675, -0.4814, -0.4393, -1.1878,  0.5852,  0.5016,  0.2002,  0.5551,
         -0.9502, -0.6856, -0.1387, -0.9701, -0